<a href="https://colab.research.google.com/github/Luis-Alatrista/TelecomX_parte1/blob/main/TelecomX_parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) EXTRACCIÓN (Ingesta de datos)

**Fuente:** `TelecomX_Data.json`

**Estrategia:**
- Lectura del JSON.
- Aplanado de estructuras anidadas con `pandas.json_normalize`.
- Normalización de nombres de columnas.


In [ ]:
import json
import pandas as pd

# Ruta al archivo JSON
DATA_JSON = "TelecomX_Data.json"

# Cargar JSON
with open(DATA_JSON, "r", encoding="utf-8") as f:
    raw = json.load(f)

# Función para detectar registros
def to_records(obj):
    if isinstance(obj, list):
        return [x for x in obj if isinstance(x, dict)]
    if isinstance(obj, dict):
        for k, v in obj.items():
            if isinstance(v, list) and v and all(isinstance(x, dict) for x in v):
                return v
        return [obj]
    return []

records = to_records(raw)
df_raw = pd.json_normalize(records, sep="_")

# Vista previa
df_raw.head()


# 2) TRANSFORMACIÓN (Limpieza y tratamiento — ETL)

**Pasos aplicados:**
1. Conversión de numéricas.
2. Eliminación de duplicados.
3. Imputación de nulos (mediana/moda).
4. Winsorización de outliers en cargos mensuales.
5. Creación de `Cuentas_Diarias = account_Charges_Monthly / 30`.


In [ ]:
import numpy as np

df = df_raw.copy()
df.columns = [c.strip().replace(" ", "_") for c in df.columns]

# Detectar columnas clave
import re
def find_col(patterns, cols):
    pat = re.compile("|".join(patterns), re.IGNORECASE)
    matches = [c for c in cols if pat.search(c)]
    return matches[0] if matches else None

col_monthly  = find_col([r"monthly.*charge", r"cargo.*mensu", r"mensual"], df.columns)
col_total    = find_col([r"total.*charge", r"cargo.*total"], df.columns)
col_tenure   = find_col([r"tenure", r"antig"], df.columns)
col_churn    = find_col([r"churn", r"evas", r"cancel"], df.columns)
col_contract = find_col([r"contract", r"contrato"], df.columns)
col_paym     = find_col([r"payment", r"pago"], df.columns)

# Tipado numérico
for c in df.columns:
    try:
        df[c] = pd.to_numeric(df[c], errors="ignore")
    except:
        pass

# Eliminar duplicados
df = df.drop_duplicates()

# Imputación
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(exclude=[np.number]).columns

for c in num_cols:
    df[c] = df[c].fillna(df[c].median())
for c in cat_cols:
    df[c] = df[c].fillna(df[c].mode()[0])

# Winsorización en MonthlyCharges
if col_monthly:
    q01, q99 = df[col_monthly].quantile([0.01, 0.99])
    df[col_monthly] = df[col_monthly].clip(q01, q99)
    df["Cuentas_Diarias"] = (df[col_monthly]/30).round(2)

df.head()


# 3) CARGA Y ANÁLISIS (Descriptivo + EDA)

Incluye estadísticos descriptivos y visualizaciones gráficas.


In [ ]:
# Estadísticos descriptivos
desc = df.describe(include="all")
desc


In [ ]:
import matplotlib.pyplot as plt

# Distribución de Churn
if col_churn and col_churn in df.columns:
    df[col_churn].value_counts().plot(kind="bar")
    plt.title("Distribución de Churn")
    plt.show()


In [ ]:
# Distribución de cargos mensuales
if col_monthly and col_monthly in df.columns:
    df[col_monthly].plot(kind="hist", bins=30)
    plt.title("Distribución de Cargos Mensuales")
    plt.show()


In [ ]:
# Distribución de Cuentas_Diarias
if "Cuentas_Diarias" in df.columns:
    df["Cuentas_Diarias"].plot(kind="hist", bins=30)
    plt.title("Distribución de Cuentas Diarias")
    plt.show()


In [ ]:
# Tenure por Churn
if col_tenure and col_churn:
    groups = [df[df[col_churn]==g][col_tenure] for g in df[col_churn].unique() if g in df[col_churn].unique()]
    plt.boxplot(groups, labels=df[col_churn].dropna().unique())
    plt.title("Tenure por Churn")
    plt.show()


In [ ]:
# Contrato vs Churn
if col_contract and col_churn:
    pd.crosstab(df[col_contract], df[col_churn], normalize="index").plot(kind="bar", stacked=True)
    plt.title("Churn por Tipo de Contrato")
    plt.show()


In [ ]:
# Método de Pago vs Churn
if col_paym and col_churn:
    pd.crosstab(df[col_paym], df[col_churn], normalize="index").plot(kind="bar")
    plt.title("Churn por Método de Pago")
    plt.show()


In [ ]:
# Matriz de correlación
num_cols = df.select_dtypes(include=[np.number]).columns
if len(num_cols) > 1:
    corr = df[num_cols].corr()
    plt.imshow(corr, cmap="coolwarm", interpolation="nearest")
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.colorbar()
    plt.title("Matriz de Correlación")
    plt.show()


# 4) INFORME FINAL

## Introducción
El churn es la evasión de clientes. El objetivo es identificar factores asociados a la pérdida de clientes en *Telecom X*.

## Limpieza y Tratamiento de Datos
- Extracción desde JSON.
- Tipado, imputación, outliers, feature engineering.

## Análisis Exploratorio de Datos
- Tasa de churn: 25.72%.
- Drivers: contrato, método de pago, tenure, cargos.

## Conclusiones e Insights
1. Contrato mes a mes concentra mayor churn.
2. Pagos automáticos reducen churn.
3. Baja antigüedad es crítica.
4. Cargos altos sin valor percibido elevan churn.

## Recomendaciones
- Migración a contratos de 1–2 años.
- Fomentar pagos automáticos.
- Bundles de valor (Soporte/Seguridad).
- Monitoreo proactivo en clientes de alto riesgo.
